In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from collections import Counter
torch.__version__

'1.5.0'

In [4]:
#读入文件
df = pd.read_csv('./adult.csv')
#salary是这个数据集最后要分类的结果
df['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [5]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [6]:
df.describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [7]:
len(df)

32561

In [8]:
#训练结果
result_var = 'income'
#分类型数据
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race','sex','native-country']
#数值型数据
cont_names = ['age', 'fnlwgt', 'education-num','capital-gain','capital-loss','hours-per-week']

In [9]:
for col in df.columns:
    if col in cat_names:
        ccol=Counter(df[col])
        print(col,len(ccol),ccol)
        print("\r\n")

workclass 9 Counter({'Private': 22696, 'Self-emp-not-inc': 2541, 'Local-gov': 2093, '?': 1836, 'State-gov': 1298, 'Self-emp-inc': 1116, 'Federal-gov': 960, 'Without-pay': 14, 'Never-worked': 7})


education 16 Counter({'HS-grad': 10501, 'Some-college': 7291, 'Bachelors': 5355, 'Masters': 1723, 'Assoc-voc': 1382, '11th': 1175, 'Assoc-acdm': 1067, '10th': 933, '7th-8th': 646, 'Prof-school': 576, '9th': 514, '12th': 433, 'Doctorate': 413, '5th-6th': 333, '1st-4th': 168, 'Preschool': 51})


occupation 15 Counter({'Prof-specialty': 4140, 'Craft-repair': 4099, 'Exec-managerial': 4066, 'Adm-clerical': 3770, 'Sales': 3650, 'Other-service': 3295, 'Machine-op-inspct': 2002, '?': 1843, 'Transport-moving': 1597, 'Handlers-cleaners': 1370, 'Farming-fishing': 994, 'Tech-support': 928, 'Protective-serv': 649, 'Priv-house-serv': 149, 'Armed-Forces': 9})


relationship 6 Counter({'Husband': 13193, 'Not-in-family': 8305, 'Own-child': 5068, 'Unmarried': 3446, 'Wife': 1568, 'Other-relative': 981})


race 

In [10]:
for col in df.columns:
    if col in cat_names:
        df[col].fillna('---')
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))
    if col in cont_names:
        df[col]=df[col].fillna(0)

In [11]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,0,77053,11,9,Widowed,0,1,4,0,0,4356,40,United-States,<=50K
1,82,4,132870,11,9,Widowed,4,1,4,0,0,4356,18,United-States,<=50K
2,66,0,186061,15,10,Widowed,0,4,2,0,0,4356,40,United-States,<=50K
3,54,4,140359,5,4,Divorced,7,4,4,0,0,3900,40,United-States,<=50K
4,41,4,264663,15,10,Separated,10,3,4,0,0,3900,40,United-States,<=50K


In [12]:
Y = df['income']
Y_label = LabelEncoder()
Y=Y_label.fit_transform(Y)
Y

array([0, 0, 0, ..., 1, 0, 0])

In [13]:
X=df.drop(columns=result_var)
X

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,0,77053,11,9,Widowed,0,1,4,0,0,4356,40,United-States
1,82,4,132870,11,9,Widowed,4,1,4,0,0,4356,18,United-States
2,66,0,186061,15,10,Widowed,0,4,2,0,0,4356,40,United-States
3,54,4,140359,5,4,Divorced,7,4,4,0,0,3900,40,United-States
4,41,4,264663,15,10,Separated,10,3,4,0,0,3900,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,4,310152,15,10,Never-married,11,1,4,1,0,0,40,United-States
32557,27,4,257302,7,12,Married-civ-spouse,13,5,4,0,0,0,38,United-States
32558,40,4,154374,11,9,Married-civ-spouse,7,0,4,1,0,0,40,United-States
32559,58,4,151910,11,9,Widowed,1,4,4,0,0,0,40,United-States


In [14]:
class tabularDataset(Dataset):
    def __init__(self, X, Y):
        self.x = X.values
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [15]:
train_ds = tabularDataset(X, Y)

In [16]:
train_ds[0]

(array([90, 0, 77053, 11, 9, 'Widowed', 0, 1, 4, 0, 0, 4356, 40,
        'United-States'], dtype=object),
 0)

In [17]:
class tabularModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(14, 500)
        self.lin2 = nn.Linear(500, 100)
        self.lin3 = nn.Linear(100, 2)
        self.bn_in = nn.BatchNorm1d(14)
        self.bn1 = nn.BatchNorm1d(500)
        self.bn2 = nn.BatchNorm1d(100)
        

    def forward(self,x_in):
        #print(x_in.shape)
        x = self.bn_in(x_in)
        x = F.relu(self.lin1(x))
        x = self.bn1(x)
        #print(x)
        
        
        x = F.relu(self.lin2(x))
        x = self.bn2(x)
        #print(x)
        
        x = self.lin3(x)
        x=torch.sigmoid(x)
        return x

In [18]:
DEVICE=torch.device("cpu")
if torch.cuda.is_available():
        DEVICE=torch.device("cuda")
print(DEVICE)

cpu


In [21]:
criterion = nn.CrossEntropyLoss()

In [22]:
model = tabularModel().to(DEVICE)
print(model)

tabularModel(
  (lin1): Linear(in_features=14, out_features=500, bias=True)
  (lin2): Linear(in_features=500, out_features=100, bias=True)
  (lin3): Linear(in_features=100, out_features=2, bias=True)
  (bn_in): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [23]:
rn=torch.rand(3,14).to(DEVICE)
model(rn)

tensor([[0.3366, 0.3982],
        [0.4719, 0.5439],
        [0.6258, 0.5256]], grad_fn=<SigmoidBackward>)

In [24]:
LEARNING_RATE=0.01
batch_size = 1024
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)